In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark dynamics

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(5)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(np.diag((.5,.5)))

q = np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten()
v = np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten()
p = M.flatf(q,v.flatten())
print("q = ", q)
print("p = ", p)

## Geodesics

In [ ]:
# 2nd order geodesic equation
from src.Riemannian import geodesic
geodesic.initialize(M)

qs = M.Exptf(q,v)
M.plot()
M.plotx(qs,v,linewidth=1.5)
plt.show()

In [ ]:
# Hamiltonian dynamics
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
M.plot()
M.plotx(qs,v)
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

# Matching of shapes:

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(10)
print(M)

from src.plotting import *

Nl = 10

# Circle representation:
theta = np.linspace(0,2*np.pi, Nl+1)
a = 0.5
b = 0.7
xc = 0
yc = 4
x = xc + a*np.sin(theta[0:Nl])
y = yc + b*np.cos(theta[0:Nl])
plt.plot(x,y,'ro')

q1 = np.vstack((x,y)).T.flatten()

# Ellipse:
a = 2
b = 0.5
cx = 0
cy = 2
xe = cx + a*np.sin(theta[0:Nl])
ye = cy + b*np.cos(theta[0:Nl])
plt.plot(xe,ye,'go')

q0 = np.vstack((xe,ye)).T.flatten()

# Initial tangent vector:
v0 = np.tile(np.array([0,1]),Nl).reshape(Nl,2)
plt.quiver(xe,ye,v0[:,0],v0[:,1], scale = 12)
v0 = v0.flatten()

plt.show()
plt.axis('equal')

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)
# example configuration
for i in range(Nl):
    diffq0 = np.diff(q0.reshape((-1,2)),axis = 0)
    diffq1 = np.diff(q1.reshape((-1,2)),axis = 0)
foo1 = np.vstack([diffq0,diffq1])
ks = np.mean(abs(foo1), axis = 0)
print(ks)

M.k_sigma.set_value(np.diag((ks[0],ks[1])))

from src.Riemannian import geodesic
geodesic.initialize(M)
from src.Riemannian import Log
Log.initialize(M)

In [ ]:
# Matching:
start = time.time()
v1 = M.Logf(q0,q1,v0)
print(v1[1])
print("time = ", time.time() - start)

In [ ]:
qs = M.Exptf(q0,v1[0])
M.plot()
M.plotx(qs,linewidth=1.5)

plt.plot(x,y,'ro')
plt.plot(xe,ye,'go')
plt.show()

# Grid transformation:

In [ ]:
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
p = M.flatf(q0,v1[0].flatten())
qsv = M.Exp_Hamiltoniantf(q0,p).T
M.plot()
M.plotx(qsv,v1[0])
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q0,p)
psv = qps[:,1,:]

In [ ]:
# Deformation of grid:
def ode_Ham_advect2(q,p,t,x):
     dxt = T.tensordot(M.gsharp(q),p,(1,0)).reshape((-1,2))
     return dxt

def ode_Ham_advect(q,p,t,x):
     dxt = T.tensordot(M.K(x,q.reshape((-1,2))),p,(1,0)).reshape((-1,2))
     return dxt

xs = T.matrix() # point to be advected with flow
qt = T.matrix() # fixed flow, q part
pt = T.matrix() # fixed flow, p part
Ham_advect = lambda xs,qt,pt: integrate(ode_Ham_advect,xs,qt,pt)
Ham_advectf = theano.function([xs,qt,pt], Ham_advect(xs,qt,pt))

Ham_advect2 = lambda xs,qt,pt: integrate(ode_Ham_advect2,xs,qt,pt)
Ham_advect2f = theano.function([xs,qt,pt], Ham_advect2(xs,qt,pt))

In [ ]:
def d2zip(grid):
    return np.dstack(grid).reshape([-1,2])

def d2unzip(points,Nx,Ny):
    return np.array([points[:,0].reshape(Nx,Ny),points[:,1].reshape(Nx,Ny)])

def getGrid(xmin,xmax,ymin,ymax,xres=None,yres=None,xpts=None,ypts=None):
        """
        Make regular grid
        Grid spacing is determined either by (x|y)res or (x|y)pts
        """

        if xres:
            xd = xres
        elif xpts:
            xd = np.complex(0,xpts)
        else:
            assert(False)
        if yres:
            yd = yres
        elif ypts:
            yd = np.complex(0,ypts)
        else:
            assert(False)

        grid = np.mgrid[xmin:xmax:xd,ymin:ymax:yd]
        Nx = grid.shape[1]
        Ny = grid.shape[2]

        return (d2zip(grid),Nx,Ny)

In [ ]:
def plotGrid(grid,Nx,Ny,coloring=True):
    """
    Plot grid
    """

    xmin = grid[:,0].min(); xmax = grid[:,0].max()
    ymin = grid[:,1].min(); ymax = grid[:,1].max()
    border = .5*(0.2*(xmax-xmin)+0.2*(ymax-ymin))

    grid = d2unzip(grid,Nx,Ny)

    color = 0.75
    colorgrid = np.full([Nx,Ny],color)
    cm = plt.cm.get_cmap('gray')
    if coloring:
        cm = plt.cm.get_cmap('coolwarm')
        hx = (xmax-xmin) / (Nx-1)
        hy = (ymax-ymin) / (Ny-1)
        for i,j in itertools.product(range(Nx),range(Ny)):
            p = grid[:,i,j]
            xs = np.empty([0,2])
            ys = np.empty([0,2])
            if 0 < i:
                xs = np.vstack((xs,grid[:,i,j]-grid[:,i-1,j],))
            if i < Nx-1:
                xs = np.vstack((xs,grid[:,i+1,j]-grid[:,i,j],))
            if 0 < j:
                ys = np.vstack((ys,grid[:,i,j]-grid[:,i,j-1],))
            if j < Ny-1:
                ys = np.vstack((ys,grid[:,i,j+1]-grid[:,i,j],))

            Jx = np.mean(xs,0) / hx
            Jy = np.mean(ys,0) / hy
            J = np.vstack((Jx,Jy,)).T

            A = .5*(J+J.T)-np.eye(2)
            CSstrain = np.log(np.trace(A*A.T))
            logdetJac = np.log(sp.linalg.det(J))
            colorgrid[i,j] = logdetJac

        cmin = np.min(colorgrid)
        cmax = np.max(colorgrid)
        f = 2*np.max((np.abs(cmin),np.abs(cmax),.5))
        colorgrid = colorgrid / f + 0.5

        print("mean color: ", np.mean(colorgrid))

        # plot lines
    for i,j in itertools.product(range(Nx),range(Ny)):
        if i < Nx-1:
            plt.plot(grid[0,i:i+2,j],grid[1,i:i+2,j],color=cm(colorgrid[i,j]), alpha = 0.7)
        if j < Ny-1:
            plt.plot(grid[0,i,j:j+2],grid[1,i,j:j+2],color=cm(colorgrid[i,j]), alpha = 0.7)

    #for i in range(0,grid.shape[1]):
    #    plt.plot(grid[0,i,:],grid[1,i,:],color)
    ## plot x lines
    #for i in range(0,grid.shape[2]):
    #    plt.plot(grid[0,:,i],grid[1,:,i],color)
    plt.xlim(xmin-border,xmax+border)
    plt.ylim(ymin-border,ymax+border)

In [ ]:
# test
qqsv = Ham_advectf(q0.reshape((-1,2)),
                    qsv,psv)[1].reshape((n_steps.eval(),-1))
#M.plotx(qqsv)
# grid
(grid,Nx,Ny) = getGrid(-4.,4.,1.,5.,xpts=50,ypts=50)
mgrid = Ham_advectf(grid,qsv,psv)[1][-1]
plotGrid(mgrid,Nx,Ny,coloring=True)

# Evolution:
for i in range(qqsv.shape[0]):
    xi = qqsv[i,:].reshape((-1,2))
    NN = xi.shape[0]
    
    for j in range(NN):
        if (i == 0) or (i == qqsv.shape[0]-1):
            plt.plot(xi[j,0],xi[j,1],'bo')
        else:
            prevx = qqsv[i-1].reshape((NN,2))
            xx = np.stack((prevx[j,:],xi[j,:]))
            plt.plot(xx[:,0],xx[:,1],'b-')

plt.show()
plt.axis('equal')

In [ ]:
# Make movie:
import os
mgrid = Ham_advectf(grid,qsv,psv)[1]
w = 40
h = 25
for k in np.arange(0,100,2)[1:]:
    qsvk = qsv[0:k,:]
    (grid,Nx,Ny) = getGrid(-4.,4.,1.,5.,xpts=50,ypts=50)
    plotGrid(mgrid[k,:,:],Nx,Ny,coloring=True)

    # Evolution:
    for i in range(qsvk.shape[0]):
        xi = qsvk[i,:].reshape((-1,2))
        NN = xi.shape[0]
    
        for j in range(NN):
            if (i == 0) or (i == qsvk.shape[0]-1):
                plt.plot(xi[j,0],xi[j,1],'bo')
            else:
                prevx = qsvk[i-1].reshape((NN,2))
                xx = np.stack((prevx[j,:],xi[j,:]))
                plt.plot(xx[:,0],xx[:,1],'b-')
    plt.axis('equal')
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIE/", 'pic%04d.png' % k-1)
    plt.savefig(filename)
    #writePictureTo(pic, filename)

In [ ]:
nb_frames = 60       # Will give 60 frames at 30 fps => movie duration : 2 sec.
  for z in range(0, nb_frames):
    pic=makeEmptyPicture(w, h)
    for x in range (0, w):
      for y in range (0, h):
        #makeColor() takes red, green, and blue (in that order) between 0 and 255
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        color = makeColor(r,g,b)
        px = getPixel(pic, x, y)
        setColor(px, color)

    # Create one frame and inject in the movie object
    filename = os.path.join(folder, 'pic%03d.png' % z)
    writePictureTo(pic, filename)
    addFrameToMovie(filename, movie)



# Brownian motions and grid change:

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(20)
print(M)

from src.plotting import *

Nl = M.N.eval()

# example configuration
M.k_sigma.set_value(np.diag((1,1)))
M.k_alpha.set_value(1)
n_steps.set_value(50)

In [ ]:
# Circle representation:
theta = np.linspace(0,2*np.pi, Nl+1)
a = 1
b = 1
xc = 0
yc = 4
x = xc + a*np.sin(theta[0:Nl])
y = yc + b*np.cos(theta[0:Nl])
plt.close()
plt.plot(x,y,'ro')
plt.show()

q0 = np.vstack((x,y)).T.flatten()

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

dwt0 = dWsf(M.dim.eval())
(ts,qs) = M.Brownian_coordsf(q0,dwt0)

#dqs = np.zeros_like(qs)
#for i in range(n_steps.eval()):
#    sde0 = M.sde_Brownian_coordsf(dwt0[i,:],i,qs[i,:])
#    dqs[i,:] = sde0[0]*dt.eval() + sde0[1]

M.plot()
M.plotx(qs)
plt.show()

In [ ]:
from src.utils import *

# Brownian motion as dx_t = K*dW_t:
def sde_KBrown(dwt,t,q):
    det = T.zeros_like(q)
    X = theano.tensor.slinalg.Cholesky()(M.gsharp(q))
    sto = T.tensordot(X,dwt,(1,0))#M.gsharp(q),dwt,(1,0))
    return (det,sto,X)
dwt = T.matrix()
q = T.vector()
BrownK = lambda q,dwt: integrate_sde(sde_KBrown,integrator_ito,q,dwt)
Brownkf = theano.function([q,dwt], BrownK(q,dwt))

In [ ]:
(tsK,qsK) = Brownkf(q0,dwt0)
M.plot()
M.plotx(qsK)
plt.show()

In [ ]:
import os
for i in range(n_steps.eval()):
    qsr = qs[i,:].reshape((M.N.eval(),2))
    qsKr = qsK[i,:].reshape((M.N.eval(),2))
    plt.plot(qsr[:,0],qsr[:,1], 'r-')
    #plt.plot(qsr[:,0],qsr[:,1], 'ro')
    plt.plot(np.array([qsr[0,0],qsr[-1,0]]),np.array([qsr[0,1],qsr[-1,1]]),'r-')
    plt.plot(qsKr[:,0],qsKr[:,1], 'g-')
    #plt.plot(qsKr[:,0],qsKr[:,1], 'go')
    plt.plot(np.array([qsKr[0,0],qsKr[-1,0]]),
             np.array([qsKr[0,1],qsKr[-1,1]]),'g-')
    plt.axis((-4,4,0,8))
    #plt.show()
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIES/", 'pic%04d.png' % i)
    plt.savefig(filename)
    plt.close()


In [ ]:
dqs = np.vstack([qs[0,:],np.diff(qs, axis = 0)])

In [ ]:
# Momentum for each time step:
ps = np.zeros_like(qs)
for i in range(n_steps.eval()):
    ps[i,:] = np.tensordot(M.gf(qs[i,:]),dqs[i,:],(1,0))

In [ ]:
print(qs)
print(qqsv)

In [ ]:
# test
qqsv = Ham_advect2f(q0.reshape((-1,2)),
                    qs,ps)[1].reshape((n_steps.eval(),-1))
#M.plotx(qqsv)
# grid
(grid,Nx,Ny) = getGrid(-3.,3.,2.,6.,xpts=50,ypts=50)
mgrid = Ham_advectf(grid,qs,ps)[1][-1]
plotGrid(mgrid,Nx,Ny,coloring=True)

# Evolution:
for i in range(qqsv.shape[0]):
    xi = qqsv[i,:].reshape((-1,2))
    NN = xi.shape[0]
    
    for j in range(NN):
        if (i == 0) or (i == qqsv.shape[0]-1):
            plt.plot(xi[j,0],xi[j,1],'bo')
        else:
            prevx = qqsv[i-1].reshape((NN,2))
            xx = np.stack((prevx[j,:],xi[j,:]))
            plt.plot(xx[:,0],xx[:,1],'b-')

plt.show()
plt.axis('equal')

In [ ]:
import os
qqsv = Ham_advectf(q0.reshape((-1,2)),
                    qs,ps)[1].reshape((n_steps.eval(),-1))
(grid,Nx,Ny) = getGrid(-3.,3.,2.,7.,xpts=30,ypts=30)
mgrid = Ham_advectf(grid,qs,ps)[1]
w = 40
h = 25
plt.close()
for k in range(50):
    plt.close()
    qsvk = qqsv[0:k,:]
    #(grid,Nx,Ny) = getGrid(-4.,4.,1.,5.,xpts=30,ypts=30)
    plotGrid(mgrid[k,:,:],Nx,Ny,coloring=True)

    # Evolution:
    for i in range(qsvk.shape[0]):
        xi = qsvk[i,:].reshape((-1,2))
        NN = xi.shape[0]
    
        for j in range(NN):
            if (i == 0) or (i == qsvk.shape[0]-1):
                plt.plot(xi[j,0],xi[j,1],'bo')
            else:
                prevx = qsvk[i-1].reshape((NN,2))
                xx = np.stack((prevx[j,:],xi[j,:]))
                plt.plot(xx[:,0],xx[:,1],'b-')
    plt.axis('equal')
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIES/", 'picg%04d.png' % k)
    plt.savefig(filename)
    #writePictureTo(pic, filename)

# Brownian motion on SO(3)

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
# setup for testing different versions of stochastic dynamics
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])

from src.group import invariant_metric
invariant_metric.initialize(G)

p = G.sharppsif(q,v)
mu = G.sharpVf(v)
print(p)
print(mu)

from src.group import energy
energy.initialize(G)

In [ ]:
# Brownian motion
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.diag(np.array([1,1,1])))

srng.seed(420)
dwt0 = dWsf(G.dim.eval())

# srng.seed(422)
(ts,gsv) = G.Brownian_invf(g,dwt0)
newfig()
G.plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()
#plt.savefig('stocso32.pdf')

In [ ]:
import os
for i in range(n_steps.eval()):
    newfig()
    G.plotg(gsv[i,:,:])
    #plt.show()
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic%04d.png' % i)
    plt.savefig(filename)
    plt.close()

for i in np.arange(1,100):
    #newfig()
    M.plot()
    x = np.array([0,0,1])
    M.plotx(M.actsf(gsv[0:i,:,:].transpose((1,2,0)),x).T)
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pics%04d.png' % i)
    plt.savefig(filename)
    plt.close()

In [ ]:
G.sigma.set_value(np.diag(np.array([2,1,1])))

#srng.seed(422)
(ts,gsv) = G.Brownian_invf(g,dwt0)
newfig()
G.plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()

In [ ]:
import os
for i in range(n_steps.eval()):
    newfig()
    G.plotg(gsv[i,:,:])
    #plt.show()
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic211%04d.png' % i)
    plt.savefig(filename)
    plt.close()

for i in np.arange(1,100):
    #newfig()
    M.plot()
    x = np.array([0,0,1])
    M.plotx(M.actsf(gsv[0:i,:,:].transpose((1,2,0)),x).T)
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic211s%04d.png' % i)
    plt.savefig(filename)
    plt.close()

In [ ]:
G.sigma.set_value(np.diag(np.array([1,2,1])))

#srng.seed(422)
(ts,gsv) = G.Brownian_invf(g,dwt0)
newfig()
G.plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()

In [ ]:
import os
for i in range(n_steps.eval()):
    newfig()
    G.plotg(gsv[i,:,:])
    #plt.show()
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic121%04d.png' % i)
    plt.savefig(filename)
    plt.close()

for i in np.arange(1,100):
    #newfig()
    M.plot()
    x = np.array([0,0,1])
    M.plotx(M.actsf(gsv[0:i,:,:].transpose((1,2,0)),x).T)
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic121s%04d.png' % i)
    plt.savefig(filename)
    plt.close()

In [ ]:
G.sigma.set_value(np.diag(np.array([1,1,2])))

#srng.seed(422)
(ts,gsv) = G.Brownian_invf(g,dwt0)
newfig()
G.plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()

In [ ]:
import os
for i in range(n_steps.eval()):
    newfig()
    G.plotg(gsv[i,:,:])
    #plt.show()
    # Create one frame and inject in the movie object
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic112%04d.png' % i)
    plt.savefig(filename)
    plt.close()

for i in np.arange(1,100):
    #newfig()
    M.plot()
    x = np.array([0,0,1])
    M.plotx(M.actsf(gsv[0:i,:,:].transpose((1,2,0)),x).T)
    filename = os.path.join("/home/line/Dropbox/MOVIE_son/", 'pic112s%04d.png' % i)
    plt.savefig(filename)
    plt.close()